In [11]:
#Defining custom loss functions for handling imbalanced dataset

with strategy.scope():
     
    tversky = keras.losses.Tversky(
    alpha=0.3,
    beta=0.7,
    reduction="sum_over_batch_size",
    name="tversky",
    axis=None,
    dtype=None )

    def iou (y_true, y_pred, smooth=1e-6):
        y_true = tf.cast(y_true, tf.float32)
        y_pred = tf.cast(y_pred > 0.5, tf.float32) 

        y_true_f = tf.reshape(y_true, [tf.shape(y_true)[0], -1])
        y_pred_f = tf.reshape(y_pred, [tf.shape(y_pred)[0], -1])

        intersection = tf.reduce_sum(y_true_f * y_pred_f, axis=1)
        union = tf.reduce_sum(y_true_f + y_pred_f, axis=1) - intersection

        iou = (intersection + smooth) / (union + smooth)
        return tf.reduce_mean(iou)

    categorical_focal_crossentropy =keras.losses.CategoricalFocalCrossentropy(
    alpha=0.25,
    gamma=2.0,
    from_logits=False,
    label_smoothing=0.0,
    axis=-1,
    reduction="sum_over_batch_size",
    name="categorical_focal_crossentropy",
    dtype=None )


    
    binary_focal_crossentropy =   tf.keras.losses.BinaryFocalCrossentropy(
    apply_class_balancing=False,
    alpha=0.25,
    gamma=2.0,
    from_logits=False,
    label_smoothing=0.0,
    axis=-1,
    reduction='sum_over_batch_size',
    name='binary_focal_crossentropy')

    sparse_categorical_crossentropy = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=False,
    ignore_class=None,
    reduction="sum_over_batch_size", 
    name="sparse_categorical_crossentropy",
    dtype=None )




    
    class_weightsb = tf.constant([0.41563786 , 2.49382716 ,5.17948718], dtype=tf.float32)
    class_weights = tf.constant([1.24691358, 3.36666667, 0.21443737, 33.66666667, 8.41666667, 11.22222222], dtype=tf.float32)

    def weighted_sparse_categorical_crossentropy(y_true, y_pred):

   
       scce = tf.keras.losses.sparse_categorical_crossentropy(y_true, y_pred)

  
       weights = tf.gather(class_weights, tf.cast(y_true, tf.int32))

    
       weighted_losses = scce * weights

    
       return tf.reduce_mean(weighted_losses)

    
    def weighted_sparse_categorical_crossentropyb(y_true, y_pred):
  
   
       scce = tf.keras.losses.sparse_categorical_crossentropy(y_true, y_pred)

   
       weightsb = tf.gather(class_weightsb, tf.cast(y_true, tf.int32))

   
       weighted_lossesb = scce * weightsb

  
       return tf.reduce_mean(weighted_lossesb)

    def custom_loss(y_true, y_pred):
       bce_loss = binary_focal_crossentropy(y_true, y_pred)
       tversky_loss = tversky(y_true, y_pred)
    
       lossed = (bce_loss + tversky_loss) 
       return lossed



    
#Defining a callback for saving model weights     

    checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath='model_checkpoint.weights.h5',
    save_weights_only=True,
    save_best_only=True,
    monitor='val_loss',
    verbose= 5 )